<a href="https://colab.research.google.com/github/sayak-coding/Extract-Website-Email-and-Social-Accounts-From-Company-Name/blob/main/Extract_Official_Website%2C_All_Email_Addresses_%26_All_Social_Accounts_From_Company_Name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [ ]:
import pandas as pd
from googlesearch import search
from google.colab import files
import re
import requests
from bs4 import BeautifulSoup

#Extract Official Website From Company Name

In [ ]:
df = pd.read_excel("emb.xlsx")
# df = df.iloc[6001:7120,0:1]
df

In [ ]:
def get_company_website(company_name):
    query = f"{company_name} official website"
    try:
        result = next(search(query, num = 1, stop = 1))
        #result = next(search(query, num_results=1))
        return result
    except StopIteration:
        return None

# Read the CSV file
# df = pd.read_csv('Countrywise Fashion Company Details - Sheet12.csv')  #  the actual path to your CSV file
df = df
# Extract websites
websites = []

for company_name in df["Company_name"]:
    website = get_company_website(company_name)
    websites.append(website)

# Add the websites as a new column
df['Website'] = websites

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_file.csv', index=False)  # desired output filename

#Extract Emails From Websites

In [ ]:
# Define a function to extract email addresses from a webpage
def extract_emails_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:  # Check if the request was successful
        content = response.text
        emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', content)
        return emails
    else:
        return []

# List of websites
websites = [
    'https://india.highcommission.gov.au/',
'https://mumbai.consulate.gov.au/'
]

# Initialize lists to store website and email data
website_list = []
email_list = []

# Iterate through the list of websites and extract emails
for website in websites:
    emails = extract_emails_from_url(website)
    if emails:  # If emails are found
        for email in emails:
            website_list.append(website)
            email_list.append(email)

# Create a DataFrame
data = {'Website': website_list, 'Email': email_list}
df = pd.DataFrame(data)

# Save DataFrame to Excel
df.to_excel('emails.xlsx', index=False)
print("Data saved to 'emails.xlsx' file.")

#For Merging Website And Email Address Files

In [ ]:
# Read the first Excel file
df1 = pd.read_csv("website5.csv")

# Read the second Excel file
df2 = pd.read_csv("email_results (1).csv")

# Merge the two DataFrames based on the "Website" column
merged_df = pd.merge(df1, df2, on="Website", how="outer")

# Save the merged DataFrame to a new Excel file
merged_df.to_excel("merged_file.xlsx", index=False)

#Extracting Both Official Website And All Email Adreesses Together

In [ ]:
def get_company_website(company_name):
    query = f"{company_name} official website"
    try:
        result = next(search(query, num=1, stop=1))
        return result
    except StopIteration:
        return None

def extract_emails_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            content = response.text
            emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', content)
            return emails
        else:
            return []
    except requests.exceptions.SSLError:
        return []

# Read the CSV file
df = pd.read_excel('emb.xlsx')  # Adjust the path to your CSV file

# Initialize lists to store website and email data
website_list = []
email_list = []

# Iterate through the list of companies and extract websites and emails
for company_name in df["Company_name"]:
    website = get_company_website(company_name)
    if website:
        emails = extract_emails_from_url(website)
        website_list.append(website)
        if emails:
            email_list.append(", ".join(emails))
        else:
            email_list.append(None)
    else:
        website_list.append(None)
        email_list.append(None)

# Add the website and email data as new columns
df['Website'] = website_list
df['Email'] = email_list

# Save the updated DataFrame to an Excel file
df.to_excel('company_data_with_emails.xlsx', index=False)
print("Data saved to 'company_data_with_emails.xlsx' file.")

# Extracting Social accounts (Linkedin, Facebook, Twitter, Instagram) From Official Website

In [ ]:
def fetch_social_media_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    social_media_links = {}

    # Define patterns to match social media URLs
    social_media_patterns = {
        'Facebook': r'https?://(?:www\.)?facebook\.com/[\w.-]+',
        'Twitter': r'https?://(?:www\.)?twitter\.com/[\w.-]+',
        'Instagram': r'https?://(?:www\.)?instagram\.com/[\w.-]+',
        'Linkedin': r'https?://(?:www\.)?linkedin\.com/company/[\w.-]+'
        # Add more patterns for other social media platforms if needed
    }

    for platform, pattern in social_media_patterns.items():
        matches = soup.find_all('a', href=re.compile(pattern, re.IGNORECASE))
        if matches:
            # Join the links into a single string
            social_media_links[platform] = ', '.join(link['href'] for link in matches)

    return social_media_links

# List of URLs
urls = [
    "https://centurymedia360.com/",
    "https://kpohealth.com/",
    # Add more URLs here
]

# Initialize an empty list to store data
data = []

for url in urls:
    social_media_links = fetch_social_media_links(url)
    if social_media_links:
        # Add website URL to the dictionary
        social_media_links['Website'] = url
        data.append(social_media_links)

# Create DataFrame from the data list
df = pd.DataFrame(data)

# Reorder columns as per your requirement
df = df[['Website', 'Facebook', 'Twitter', 'Instagram', 'Linkedin']]

# Save DataFrame to Excel file
df.to_excel('social_media_accounts.xlsx', index=False)

print("Social media account data saved to social_media_accounts.xlsx successfully.")

Social media account data saved to social_media_accounts.xlsx successfully.


# Extracting All Social Media Links And Emails From The Website Name

In [ ]:
def get_company_website(company_name):
    query = f"{company_name} official website"
    try:
        result = next(search(query, num=1, stop=1))
        return result
    except StopIteration:
        return None

def extract_emails_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            content = response.text
            emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', content)
            return emails
        else:
            return []
    except requests.exceptions.SSLError:
        return []

def fetch_social_media_links(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            response.raise_for_status()
        else:
            print(f"Ignoring {url} due to status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    social_media_links = {}

    # Define patterns to match social media URLs
    social_media_patterns = {
        'Facebook': r'https?://(?:www\.)?facebook\.com/[\w.-]+',
        'Twitter': r'https?://(?:www\.)?twitter\.com/[\w.-]+',
        'Instagram': r'https?://(?:www\.)?instagram\.com/[\w.-]+',
        'Linkedin': r'https?://(?:www\.)?linkedin\.com/company/[\w.-]+'
        # Add more patterns for other social media platforms if needed
    }

    for platform, pattern in social_media_patterns.items():
        matches = soup.find_all('a', href=re.compile(pattern, re.IGNORECASE))
        if matches:
            # Join the links into a single string
            social_media_links[platform] = ', '.join(link['href'] for link in matches)

    return social_media_links

# Read the Excel file with website names
df = pd.read_excel('company_test.xlsx')  # Adjust the path to your Excel file

# Initialize lists to store website, email, and social media data
website_list = []
email_list = []
social_media_data = []

# Iterate through the list of companies and extract websites, emails, and social media accounts
for company_name in df["Company_name"]:
    website = get_company_website(company_name)
    if website:
        emails = extract_emails_from_url(website)
        social_media_links = fetch_social_media_links(website)

        website_list.append(website)

        if emails:
            email_list.append(", ".join(emails))
        else:
            email_list.append(None)

        if social_media_links:
            social_media_data.append(social_media_links)
        else:
            social_media_data.append(None)
    else:
        website_list.append(None)
        email_list.append(None)
        social_media_data.append(None)

# Create DataFrame from the data lists
df['Website'] = website_list
df['Email'] = email_list

# Extract social media links into separate columns
for platform in ['Facebook', 'Twitter', 'Instagram', 'Linkedin']:
    df[platform] = [data[platform] if data and platform in data else 'Not found' for data in social_media_data]

# Reorder columns as per your requirement
df = df[['Company_name', 'Website', 'Email', 'Facebook', 'Twitter', 'Instagram', 'Linkedin']]

# Save the updated DataFrame to an Excel file
df.to_excel('company_data_with_emails_and_social_media.xlsx', index=False)
print("Data saved to 'company_data_with_emails_and_social_media.xlsx' file.")